In [1]:
from datasets.gait_fc_dataset import GaitFCDataset
from models.gait_fc import GaitFC
import torch
from torch.nn import functional as F

In [2]:
train_dataset = GaitFCDataset('gait_fc_dataset/train')
val_dataset = GaitFCDataset('gait_fc_dataset/test')

In [3]:
print(len(train_dataset))
print(len(val_dataset))

15849
7970


In [4]:
batch_size = 32
epochs = 5
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [5]:
device = torch.device('cpu')
model = GaitFC()
model.to(device)
model.train()

GaitFC(
  (gei_encoder): MobileFaceNet(
    (layers): ModuleList(
      (0): ConvBlock(
        (layers): Sequential(
          (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): PReLU(num_parameters=64)
        )
      )
      (1): ConvBlock(
        (layers): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): PReLU(num_parameters=64)
        )
      )
      (2): DepthWise(
        (layers): Sequential(
          (0): ConvBlock(
            (layers): Sequential(
              (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): PReLU(num_param

In [6]:

margin = 1.
triplet_loss = torch.nn.TripletMarginWithDistanceLoss(distance_function=lambda x, y: margin - F.cosine_similarity(x, y))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

In [7]:
start_epoch = 0
print('start training !')
for epoch in range(start_epoch, epochs):
    model.train()
    losses = []
    iter = 0
    for batch_idx, (a_v, a_gei, p_v, p_gei, n_v, n_gei) in enumerate(train_loader):
        a_v = a_v.to(device).float()
        p_v = p_v.to(device).float()
        n_v = n_v.to(device).float()

        a_gei = a_gei.to(device).float()
        p_gei = p_gei.to(device).float()
        n_gei = n_gei.to(device).float()

        anchor_out = model(a_v, a_gei)
        positive_out = model(p_v, p_gei)
        negative_out = model(n_v, n_gei)

        output = triplet_loss(anchor_out, positive_out, negative_out)
        output.backward()
        optimizer.step()


        print(f'Epoch: [%d]/[%d] Training, %.2f%%,{iter}/{len(train_dataset)}, Loss={output.item()}' % (epoch, epochs, iter*100/len(train_dataset)))
        iter += len(a_v)
        losses.append(output.item())
    print(sum(losses)/len(losses))


start training !


C:\Users\Cuong Tran\AppData\Roaming\Python\Python38\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch: [0]/[5] Training, 0.00%,0/15849, Loss=0.9419510364532471
Epoch: [0]/[5] Training, 0.20%,32/15849, Loss=0.9393965601921082
Epoch: [0]/[5] Training, 0.40%,64/15849, Loss=0.9136041402816772
Epoch: [0]/[5] Training, 0.61%,96/15849, Loss=0.8482068181037903
Epoch: [0]/[5] Training, 0.81%,128/15849, Loss=0.8986294865608215
Epoch: [0]/[5] Training, 1.01%,160/15849, Loss=0.863192081451416
Epoch: [0]/[5] Training, 1.21%,192/15849, Loss=0.9207472801208496
Epoch: [0]/[5] Training, 1.41%,224/15849, Loss=0.7817184925079346
Epoch: [0]/[5] Training, 1.62%,256/15849, Loss=0.9888633489608765
Epoch: [0]/[5] Training, 1.82%,288/15849, Loss=0.7176206111907959
Epoch: [0]/[5] Training, 2.02%,320/15849, Loss=0.8510199785232544
Epoch: [0]/[5] Training, 2.22%,352/15849, Loss=0.6970118284225464
Epoch: [0]/[5] Training, 2.42%,384/15849, Loss=0.7988058924674988
Epoch: [0]/[5] Training, 2.62%,416/15849, Loss=0.8665611743927002
Epoch: [0]/[5] Training, 2.83%,448/15849, Loss=0.8706896305084229
Epoch: [0]/[5] T

In [8]:
start_epoch = 20
epochs = 40
print('start training !')
for epoch in range(start_epoch, 20):
    model.train()
    losses = []
    iter = 0
    for batch_idx, (a_v, a_gei, p_v, p_gei, n_v, n_gei) in enumerate(train_loader):
        a_v = a_v.to(device).float()
        p_v = p_v.to(device).float()
        n_v = n_v.to(device).float()

        a_gei = a_gei.to(device).float()
        p_gei = p_gei.to(device).float()
        n_gei = n_gei.to(device).float()

        anchor_out = model(a_v, a_gei)
        positive_out = model(p_v, p_gei)
        negative_out = model(n_v, n_gei)

        output = triplet_loss(anchor_out, positive_out, negative_out)
        output.backward()
        optimizer.step()


        print(f'Epoch: [%d]/[%d] Training, %.2f%%,{iter}/{len(train_dataset)}, Loss={output.item()}' % (epoch, epochs, iter*100/len(train_dataset)))
        iter += len(a_v)
        losses.append(output.item())
    print(sum(losses)/len(losses))

start training !
Epoch: [5]/[5] Training, 0.00%,0/15849, Loss=0.41262370347976685
Epoch: [5]/[5] Training, 0.20%,32/15849, Loss=0.26523879170417786
Epoch: [5]/[5] Training, 0.40%,64/15849, Loss=0.32151857018470764
Epoch: [5]/[5] Training, 0.61%,96/15849, Loss=0.37188735604286194
Epoch: [5]/[5] Training, 0.81%,128/15849, Loss=0.2588939368724823
Epoch: [5]/[5] Training, 1.01%,160/15849, Loss=0.32780224084854126
Epoch: [5]/[5] Training, 1.21%,192/15849, Loss=0.25064101815223694
Epoch: [5]/[5] Training, 1.41%,224/15849, Loss=0.3096005320549011
Epoch: [5]/[5] Training, 1.62%,256/15849, Loss=0.49619153141975403
Epoch: [5]/[5] Training, 1.82%,288/15849, Loss=0.2911113500595093
Epoch: [5]/[5] Training, 2.02%,320/15849, Loss=0.2939125895500183
Epoch: [5]/[5] Training, 2.22%,352/15849, Loss=0.2927486002445221
Epoch: [5]/[5] Training, 2.42%,384/15849, Loss=0.35653868317604065
Epoch: [5]/[5] Training, 2.62%,416/15849, Loss=0.32300105690956116
Epoch: [5]/[5] Training, 2.83%,448/15849, Loss=0.293865

In [12]:
torch.save(model.state_dict(), 'src/gait_fc_epoch_20.pt')